In [1]:
// This is all just boilerplate, nothing to see here :).
import * as tslab from 'tslab';
import rimraf from 'rimraf';
try {
    rimraf.sync("./default")
} catch(e) {
    // do nothing here
}


# Using Tupelo for Supply Chain

This is a small demonstration of using Tupelo for supply chain. All supply chains are different and so this will be an idealized version.

We'll model this out as a shipment of coffee from an amazing collective of womens coffee farmers and roasters through to a consumer in Germany.

The basics are:

* The beans are tended in the field by Farmers
* The beans are harvested by Farmers
* The beans are roasted by Roaster
* The beans are collected into a shipping container by Shippers
* The beans are displayed by a Retailer
* The beans are purchased by a Consumer

In [2]:
import { ChainTree, Community, EcdsaKey, setDataTransaction, setOwnershipTransaction, establishTokenTransaction, mintTokenTransaction, sendTokenTransaction, receiveTokenTransaction, receiveTokenTransactionFromPayload } from 'tupelo-wasm-sdk'
import { v4 as uuidv4 } from 'uuid';


## Networking & Community
The Community object is how the local client talks to the network. You'll need one of these to play your transactions. There's a default (currently the testnet) provided for you.


In [3]:
const community = await Community.getDefault()

## Identity

All of this starts with identity. Every organization and user will have a key and a ChainTree. Let's go create those now.

In [4]:
let actorNames = ["farm", "roaster", "shipper", "retailer"]
let actors:{[key:string]:ChainTree} = {}
await Promise.all(actorNames.map(async (name:string)=> {
    let key = await EcdsaKey.generate()
    let tree = await ChainTree.newEmptyTree(community.blockservice, key)
    actors[name] = tree
}))
console.log(actors)

{
  farm: ChainTree {
    tip: CID(bafyreihfgyerwu7q3q3hlvaylvyf2lshpopkytdmt62vkqiykoi35pncva),
    dagStore: IPLDResolver {
      bs: [WrappedBlockService],
      resolvers: [Object],
      loadFormat: [Function (anonymous)]
    },
    key: EcdsaKey { publicKey: [Uint8Array], privateKey: [Uint8Array] },
    store: WrappedBlockService { blockservice: [BlockService] }
  },
  shipper: ChainTree {
    tip: CID(bafyreiapplhost4otdxd4mauvfz2eimaq6izpdonmwpuiorlefkwjjjg4e),
    dagStore: IPLDResolver {
      bs: [WrappedBlockService],
      resolvers: [Object],
      loadFormat: [Function (anonymous)]
    },
    key: EcdsaKey { publicKey: [Uint8Array], privateKey: [Uint8Array] },
    store: WrappedBlockService { blockservice: [BlockService] }
  },
  roaster: ChainTree {
    tip: CID(bafyreieulcosjuz55kxuwa3z7djz5s2o7v4dhzctcnsbmei7kgpwju7cd4),
    dagStore: IPLDResolver {
      bs: [WrappedBlockService],
      resolvers: [Object],
      loadFormat: [Function (anonymous)]
    },
    key: Ecd

## Objects
In any supply chain there are a variety of objects and you'll want to model them to your domain. In the coffee case we have:

* A field
* A basket of beans
* A shipping container
* A package of coffee

We'll start with the field and update it with some information about the conditions.

In [5]:
const fieldKey = await EcdsaKey.generate()
const fieldTree = await ChainTree.newEmptyTree(community.blockservice, fieldKey)

await community.playTransactions(fieldTree,[
    setDataTransaction(`updates/2020-04-17`, "rain"),
    setDataTransaction("description", "The most organic field you've ever seen"),
])
console.log((await fieldTree.resolveData("updates")).value)

{ '2020-04-17': 'rain' }


Now we'll harvest the beans

In [6]:
await community.playTransactions(fieldTree, [setDataTransaction(`updates/2020-04-18`, "harvest")])
console.log((await fieldTree.resolveData("updates")).value)

{ '2020-04-17': 'rain', '2020-04-18': 'harvest' }


Now the farmers create a 60kg sack of green beans from the harvest. The farm will own this ChainTree so we'll do a new type of transaction: `setOwnershipTransaction`. They will also link it up to the harvesting field.


In [7]:
const greenBeanTree = await ChainTree.newEmptyTree(community.blockservice, await EcdsaKey.generate())
await community.playTransactions(greenBeanTree, [
    setOwnershipTransaction([await actors["farm"].key.address()]),
    setDataTransaction("origin", await fieldTree.id())
])
greenBeanTree.key = actors["farm"].key
console.log("ownership: ", (await greenBeanTree.resolve("tree/_tupelo/authentications")).value)
console.log("origin (field id): ", (await greenBeanTree.resolveData("origin")).value)

ownership:  [ '0x094561C123c091A4c4F5b262E08fBBD1bcb5331B' ]
origin (field id):  did:tupelo:0x4eFD62299E7d54E62e1F4D26e5CD7700C10071Ad


We're going to send it off to the roasters by changing the ownership and adding a delivery event.

In [8]:
await community.playTransactions(greenBeanTree, [
    setDataTransaction("events/2020-04-18", "delivery to roaster"),
    setOwnershipTransaction([await actors["roaster"].key.address()])])
greenBeanTree.key = actors["roaster"].key
console.log((await greenBeanTree.resolveData("events")).value)


{ '2020-04-18': 'delivery to roaster' }


The roaster roasts the sack of beans, packages them into 1kg bags and sends those bags to the shipper. We are going to use tokens to represent each 1kg of green beans becoming a 1kg packaged bag (with a QR code of the DID). In the real world we'd do one for each Kg, but to simplify here we're only going to do 5 of them.

In [9]:
await community.playTransactions(greenBeanTree, [
    setDataTransaction("events/2020-04-19", "out for delivery"),
    establishTokenTransaction("1kg", 60), // the 60 here means that only 60 of these can ever be created
    mintTokenTransaction("1kg", 60), // mint all 60 of them
    ])

// token names are always prefixed by the chaintree that created them
const canonicalName = (await greenBeanTree.id()) + ":" + "1kg"

let consumerBags = []

for (let i = 0; i < 5; i++) { // only creating 5, but would be 60 in the real world
    let bagTree = await ChainTree.newEmptyTree(community.blockservice, await EcdsaKey.generate())
    consumerBags[i] = bagTree
    let payload = await community.sendTokenAndGetPayload(greenBeanTree, sendTokenTransaction(uuidv4(), canonicalName, 1, await bagTree.id()))
    
    await community.playTransactions(bagTree, [
        setOwnershipTransaction([await actors["shipper"].key.address()]),
        receiveTokenTransactionFromPayload(payload),
        setDataTransaction("events/2020-04-19", "packaged")
    ])
    bagTree.key = actors["shipper"].key
}
// greenBeanTree.key = actors["shipper"].key

console.log((await greenBeanTree.resolveData("events")).value)



{
  '2020-04-18': 'delivery to roaster',
  '2020-04-19': 'out for delivery'
}


The shipper creates a manifest for the shipping container. In a real coffee supply chain the manifest would contain lots of the bags from above. 

In [10]:
const containerTree = await ChainTree.newEmptyTree(community.blockservice, await EcdsaKey.generate())
await community.playTransactions(containerTree, [
    setOwnershipTransaction([await actors["shipper"].key.address()]),
    setDataTransaction("manifest", await Promise.all(consumerBags.map(async (bag)=> {
        return await bag.id()
    })))
])
containerTree.key = actors["shipper"].key
console.log((await containerTree.resolveData("manifest")).value)


[
  'did:tupelo:0x3FAF459B7C3aff610A82864053B9A01BF3931E96',
  'did:tupelo:0x00A15Def0Eb4B8cd005Bc40dA3e463A716940A14',
  'did:tupelo:0xb9f64Ec5b2a96bB3E3e576aDa9ed49bb6985262C',
  'did:tupelo:0x8f51D57e0a027B40F5D7C47de94f6f60049Cd942',
  'did:tupelo:0x52a9EB752ed16A62A42D275f3eAD8981Ba314A65'
]


The retailer takes ownership of all these bags

In [11]:
const retailerAddress = await actors["retailer"].key.address()

await Promise.all(consumerBags.map(async (bag)=> {
        await community.playTransactions(bag, [
            setOwnershipTransaction([retailerAddress]),
            setDataTransaction("events/2020-04-20", "received in store")
        ])
        bag.key = actors["retailer"].key
}))
console.log("retailer has it!")

retailer has it!


And now the consumer can get a full history of the object! We'll pick one of the bags above and pretend that a consumer has scanned the QR code:

In [12]:
let consumerBag = consumerBags[0]
console.log((await consumerBag.resolveData("events")).value)
console.log((await greenBeanTree.resolveData("events")).value)
console.log((await fieldTree.resolveData("description")).value)
console.log((await fieldTree.resolveData("updates")).value)

{ '2020-04-19': 'packaged', '2020-04-20': 'received in store' }
{
  '2020-04-18': 'delivery to roaster',
  '2020-04-19': 'out for delivery'
}
The most organic field you've ever seen
{ '2020-04-17': 'rain', '2020-04-18': 'harvest' }
